In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable


INFO: pip is looking at multiple versions of google-cloud-storage to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.8/336.8 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 73.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import pandas as pd

In [2]:
# Konfigurasi database sumber
source_cred = credentials.Certificate("/work/database-sensor-iklim-litbang-firebase-admin.json")
firebase_admin.initialize_app(source_cred, {
    'databaseURL': 'https://database-sensor-iklim-litbang-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [13]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-03')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    df.index.name = 'timestamp'  # Mengatur nama indeks
    df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    print(df.head())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di jalur /auto_weather_stat/id-03.")

  timestamp                                         1731000619  \
0      data  {'dew': 24.83318, 'humidity': 98.48, 'pressure...   
1  metadata                                                NaN   

                                          1731000678  \
0  {'dew': 24.83488, 'humidity': 98.49, 'pressure...   
1                                                NaN   

                                          1731000739  \
0  {'dew': 24.83318, 'humidity': 98.48, 'pressure...   
1                                                NaN   

                                          1731000798  \
0  {'dew': 24.83148, 'humidity': 98.47, 'pressure...   
1                                                NaN   

                                          1731000859  \
0  {'dew': 24.79155, 'humidity': 98.47, 'pressure...   
1                                                NaN   

                                          1731000918  \
0  {'dew': 24.78985, 'humidity': 98.46, 'pressure...   
1            

In [8]:
# Tentukan rentang waktu (timestamp dalam Unix Time)
start_timestamp = 1733169600  # Contoh: 11 Desember 2023 00:00:00
end_timestamp =   1733176800    # Contoh: 11 Desember 2023 23:59:59

# Filter data berdasarkan rentang waktu dengan validasi kunci
filtered_data = {
    key: value
    for key, value in source_data.items()
    if key.isdigit() and start_timestamp <= int(key) <= end_timestamp
}


In [9]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
}, name='destination')

In [10]:
# Referensi ke data tujuan
dest_ref = db.reference('/auto_weather_stat/id-04', app=dest_app)

In [11]:
# Memindahkan data yang telah difilter ke database tujuan
if filtered_data:
    dest_ref.update(filtered_data)
    print("Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.")
else:
    print("Tidak ada data dalam rentang waktu yang ditentukan.")

Tidak ada data dalam rentang waktu yang ditentukan.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>